Deep Neural Network Design (for adversarial training), with Self Scaling Formulation:

Reid Zaffino (zaffino@uwindsor.ca) 2021-09

Resources from Coursera "Neural Networks and Deep Learning" Course

Self Scaling Methodology from: Djahanshahi, Hormoz., "A robust hybrid VLSI neural network architecture for a smart optical sensor." (1999). Electronic Theses and Dissertations. 737. https://scholar.uwindsor.ca/etd/737

In [1]:
import time
import numpy as np
import matplotlib.pyplot as plt
import os
from six.moves import cPickle 

np.random.seed(1)

In [2]:
dir_path =  os.path.realpath('./')
file1 = dir_path + "\cifar-10-batches-py\data_batch_1"

f = open(file1, 'rb')
datadict = cPickle.load(f,encoding='latin1')
f.close()
X = datadict["data"] 
Y = datadict['labels']

nonbin = []
for i in range (len(Y)):
    if (Y[i] != 0 and Y[i] != 1):
        nonbin.append(i)

X = np.delete(X, nonbin, axis = 0)
Y = np.delete(Y, nonbin)
#a_list = list(range(1, 1000))
#X = np.delete(X, a_list, axis = 0)
#Y = np.delete(Y, a_list)

X = (X.reshape(X.shape[0], -1)/255.).T
Y = Y.reshape(1, Y.shape[0])

train_x = X
train_y = Y

In [3]:
layers_dims = [3072, 20, 7, 5, 1] #  4-layer model

In [4]:
def initialize_parameters_deep(layer_dims):
    """
    Arguments:
    layer_dims -- python array (list) containing the dimensions of each layer in our network
    
    Returns:
    parameters -- python dictionary containing your parameters "W1", "b1", ..., "WL", "bL":
                    Wl -- weight matrix of shape (layer_dims[l], layer_dims[l-1])
                    bl -- bias vector of shape (layer_dims[l], 1)
    """
    
    np.random.seed(3)
    parameters = {}
    L = len(layer_dims) # number of layers in the network

    for l in range(1, L):
        #(≈ 2 lines of code)
        # YOUR CODE STARTS HERE
        
        parameters['W' + str(l)] = np.random.randn(layer_dims[l], layer_dims[l-1]) * 0.01
        parameters['b' + str(l)] = np.zeros((layer_dims[l], 1))
        
        # YOUR CODE ENDS HERE
        
        assert(parameters['W' + str(l)].shape == (layer_dims[l], layer_dims[l - 1]))
        assert(parameters['b' + str(l)].shape == (layer_dims[l], 1))

        
    return parameters

In [5]:
def sigmoid(Z):
    
    return np.divide(1, 1 + np.exp(-Z)), Z

In [6]:
def tanh(Z):
    
    #return Z * (Z > 0), Z
    return np.tanh(Z), Z

In [7]:
def linear_forward(A, W, b):
    """
    Implement the linear part of a layer's forward propagation.

    Arguments:
    A -- activations from previous layer (or input data): (size of previous layer, number of examples)
    W -- weights matrix: numpy array of shape (size of current layer, size of previous layer)
    b -- bias vector, numpy array of shape (size of the current layer, 1)

    Returns:
    Z -- the input of the activation function, also called pre-activation parameter 
    cache -- a python tuple containing "A", "W" and "b" ; stored for computing the backward pass efficiently
    """
    
    #(≈ 1 line of code)
    # YOUR CODE STARTS HERE
    
    Z = np.dot(W,A) + b
    
    # YOUR CODE ENDS HERE
    cache = (A, W, b)
    
    return Z, cache

In [8]:
def linear_activation_forward(A_prev, W, b, activation, ss):
    """
    Implement the forward propagation for the LINEAR->ACTIVATION layer

    Arguments:
    A_prev -- activations from previous layer (or input data): (size of previous layer, number of examples)
    W -- weights matrix: numpy array of shape (size of current layer, size of previous layer)
    b -- bias vector, numpy array of shape (size of the current layer, 1)
    activation -- the activation to be used in this layer, stored as a text string: "sigmoid" or "tanh"

    Returns:
    A -- the output of the activation function, also called the post-activation value 
    cache -- a python tuple containing "linear_cache" and "activation_cache";
             stored for computing the backward pass efficiently
    """
    
    if activation == "sigmoid":
        #(≈ 2 lines of code)
        # YOUR CODE STARTS HERE
        
        if(ss):
            Z, linear_cache = linear_forward(A_prev, W, b)
            A, activation_cache = sigmoid(Z/len(A_prev))
        
        else:
            Z, linear_cache = linear_forward(A_prev, W, b)
            A, activation_cache = sigmoid(Z)
            
        # YOUR CODE ENDS HERE
    
    elif activation == "tanh":
        #(≈ 2 lines of code)
        # YOUR CODE STARTS HERE
        
        if(ss):
            Z, linear_cache = linear_forward(A_prev, W, b)
            A, activation_cache = tanh(Z/len(A_prev))
        
        else:
            Z, linear_cache = linear_forward(A_prev, W, b)
            A, activation_cache = tanh(Z)
            
        # YOUR CODE ENDS HERE
    cache = (linear_cache, activation_cache)

    return A, cache

In [9]:
def L_model_forward(X, parameters, ss):
    """
    Implement forward propagation for the [LINEAR->tanh]*(L-1)->LINEAR->SIGMOID computation
    
    Arguments:
    X -- data, numpy array of shape (input size, number of examples)
    parameters -- output of initialize_parameters_deep()
    
    Returns:
    AL -- activation value from the output (last) layer
    caches -- list of caches containing:
                every cache of linear_activation_forward() (there are L of them, indexed from 0 to L-1)
    """

    caches = []
    A = X
    L = len(parameters) // 2                  # number of layers in the neural network
    
    # Implement [LINEAR -> tanh]*(L-1). Add "cache" to the "caches" list.
    # The for loop starts at 1 because layer 0 is the input
    for l in range(1, L):
        A_prev = A 
        #(≈ 2 lines of code)
        # YOUR CODE STARTS HERE
        
        A, cache = linear_activation_forward(A_prev, parameters['W' + str(l)], parameters['b' + str(l)], "tanh", ss)
        caches.append(cache)
        
        # YOUR CODE ENDS HERE
    
    # Implement LINEAR -> SIGMOID. Add "cache" to the "caches" list.
    #(≈ 2 lines of code)
    # YOUR CODE STARTS HERE
    
    AL, cache = linear_activation_forward(A, parameters['W' + str(L)], parameters['b' + str(L)], "sigmoid", ss)
    caches.append(cache)
    
    # YOUR CODE ENDS HERE
          
    return AL, caches

In [10]:
def compute_cost(AL, Y):
    """
    Implement the cost function defined by equation (7).

    Arguments:
    AL -- probability vector corresponding to your label predictions, shape (1, number of examples)
    Y -- true "label" vector (for example: containing 0 if non-cat, 1 if cat), shape (1, number of examples)

    Returns:
    cost -- cross-entropy cost
    """
    
    m = Y.shape[1]

    # Compute loss from aL and y.
    # (≈ 1 lines of code)
    # YOUR CODE STARTS HERE
    
    cost = (-1/m)*np.sum(np.multiply(np.log(AL), Y) + np.multiply(np.log(1 - AL), 1 - Y))
    
    # YOUR CODE ENDS HERE
    
    cost = np.squeeze(cost)      # To make sure your cost's shape is what we expect (e.g. this turns [[17]] into 17).

    
    return cost

In [11]:
def linear_backward(dZ, cache):
    """
    Implement the linear portion of backward propagation for a single layer (layer l)

    Arguments:
    dZ -- Gradient of the cost with respect to the linear output (of current layer l)
    cache -- tuple of values (A_prev, W, b) coming from the forward propagation in the current layer

    Returns:
    dA_prev -- Gradient of the cost with respect to the activation (of the previous layer l-1), same shape as A_prev
    dW -- Gradient of the cost with respect to W (current layer l), same shape as W
    db -- Gradient of the cost with respect to b (current layer l), same shape as b
    """
    A_prev, W, b = cache
    m = A_prev.shape[1]

    ### START CODE HERE ### (≈ 3 lines of code)
    # YOUR CODE STARTS HERE
    
    dW = (1/m)*np.dot(dZ, A_prev.T)
    db = (1/m)*np.sum(dZ, axis = 1, keepdims = True)
    dA_prev = np.dot(W.T, dZ)
    
    # YOUR CODE ENDS HERE
    
    return dA_prev, dW, db

In [12]:
def sigmoid_backward(dA, activation_cache):
    
    return np.multiply(dA, np.multiply(np.divide(1, 1 + np.exp(-activation_cache)), 1 - np.divide(1, 1 + np.exp(-activation_cache))))

In [13]:
def tanh_backward(dA, activation_cache):
    
    #return np.multiply(dA, (activation_cache > 0) * 1)
    return np.multiply(dA, 1 - np.tanh(activation_cache)**2)

In [14]:
def linear_activation_backward(dA, cache, activation):
    """
    Implement the backward propagation for the LINEAR->ACTIVATION layer.
    
    Arguments:
    dA -- post-activation gradient for current layer l 
    cache -- tuple of values (linear_cache, activation_cache) we store for computing backward propagation efficiently
    activation -- the activation to be used in this layer, stored as a text string: "sigmoid" or "tanh"
    
    Returns:
    dA_prev -- Gradient of the cost with respect to the activation (of the previous layer l-1), same shape as A_prev
    dW -- Gradient of the cost with respect to W (current layer l), same shape as W
    db -- Gradient of the cost with respect to b (current layer l), same shape as b
    """
    linear_cache, activation_cache = cache
    
    if activation == "tanh":
        #(≈ 2 lines of code)
        # YOUR CODE STARTS HERE
        
        dZ =  tanh_backward(dA, activation_cache)
        #print(activation_cache)
        dA_prev, dW, db = linear_backward(dZ, linear_cache)
        
        # YOUR CODE ENDS HERE
        
    elif activation == "sigmoid":
        #(≈ 2 lines of code)
        # YOUR CODE STARTS HERE
        
        dZ = sigmoid_backward(dA, activation_cache)
        dA_prev, dW, db = linear_backward(dZ, linear_cache)
        
        # YOUR CODE ENDS HERE
    
    return dA_prev, dW, db

In [15]:
def L_model_backward(AL, Y, caches):
    """
    Implement the backward propagation for the [LINEAR->tanh] * (L-1) -> LINEAR -> SIGMOID group
    
    Arguments:
    AL -- probability vector, output of the forward propagation (L_model_forward())
    Y -- true "label" vector (containing 0 if non-cat, 1 if cat)
    caches -- list of caches containing:
                every cache of linear_activation_forward() with "tanh" (it's caches[l], for l in range(L-1) i.e l = 0...L-2)
                the cache of linear_activation_forward() with "sigmoid" (it's caches[L-1])
    
    Returns:
    grads -- A dictionary with the gradients
             grads["dA" + str(l)] = ... 
             grads["dW" + str(l)] = ...
             grads["db" + str(l)] = ... 
    """
    grads = {}
    L = len(caches) # the number of layers
    m = AL.shape[1]
    #print("This stuff: ")
    #print(Y.shape)
    Y = Y.reshape(AL.shape) # after this line, Y is the same shape as AL
    #print("This stuff 2.0: ")
    #print(Y.shape)

    
    # Initializing the backpropagation
    #(1 line of code)
    # YOUR CODE STARTS HERE
    
    dAL = -(np.divide(Y, AL) - np.divide(1 - Y, 1 - AL))
    
    # YOUR CODE ENDS HERE
    
    # Lth layer (SIGMOID -> LINEAR) gradients. Inputs: "dAL, current_cache". Outputs: "grads["dAL-1"], grads["dWL"], grads["dbL"]
    #(approx. 5 lines)
    # YOUR CODE STARTS HERE
    
    current_cache = caches[L-1]
    dA_prev_temp, dW_temp, db_temp = linear_activation_backward(dAL, current_cache, "sigmoid")
    grads["dA" + str(L-1)] = dA_prev_temp
    grads["dW" + str(L)] = dW_temp
    grads["db" + str(L)] = db_temp
    
    # YOUR CODE ENDS HERE
    
    # Loop from l=L-2 to l=0
    for l in reversed(range(L-1)):
        # lth layer: (tanh -> LINEAR) gradients.
        # Inputs: "grads["dA" + str(l + 1)], current_cache". Outputs: "grads["dA" + str(l)] , grads["dW" + str(l + 1)] , grads["db" + str(l + 1)] 
        #(approx. 5 lines)
        
        # YOUR CODE STARTS HERE
        
        current_cache = caches[l]
        dA_prev_temp, dW_temp, db_temp = linear_activation_backward(dA_prev_temp, current_cache, "tanh")
        grads["dA" + str(l)] = dA_prev_temp
        grads["dW" + str(l + 1)] = dW_temp
        grads["db" + str(l + 1)] = db_temp
        
        # YOUR CODE ENDS HERE

    return grads

In [16]:
def update_parameters(params, grads, learning_rate):
    """
    Update parameters using gradient descent
    
    Arguments:
    params -- python dictionary containing your parameters 
    grads -- python dictionary containing your gradients, output of L_model_backward
    
    Returns:
    parameters -- python dictionary containing your updated parameters 
                  parameters["W" + str(l)] = ... 
                  parameters["b" + str(l)] = ...
    """
    parameters = params.copy()
    L = len(parameters) // 2 # number of layers in the neural network

    # Update rule for each parameter. Use a for loop.
    #(≈ 2 lines of code)
    for l in range(L):
        # YOUR CODE STARTS HERE
        
        parameters["W" + str(l+1)] = parameters["W" + str(l+1)] - learning_rate*grads["dW" + str(l+1)]
        parameters["b" + str(l+1)] = parameters["b" + str(l+1)] - learning_rate*grads["db" + str(l+1)]
        
        # YOUR CODE ENDS HERE
        
    return parameters

In [17]:
def L_layer_model(X, Y, layers_dims, learning_rate = 0.5, num_iterations = 3000, print_cost=False, ss=False):
    """
    Implements a L-layer neural network: [LINEAR->tanh]*(L-1)->LINEAR->SIGMOID.
    
    Arguments:
    X -- data, numpy array of shape (num_px * num_px * 3, number of examples)
    Y -- true "label" vector (containing 0 if cat, 1 if non-cat), of shape (1, number of examples)
    layers_dims -- list containing the input size and each layer size, of length (number of layers + 1).
    learning_rate -- learning rate of the gradient descent update rule
    num_iterations -- number of iterations of the optimization loop
    print_cost -- if True, it prints the cost every 100 steps
    
    Returns:
    parameters -- parameters learnt by the model. They can then be used to predict.
    """

    np.random.seed(1)
    costs = []                         # keep track of cost
    
    # Parameters initialization.
    #(≈ 1 line of code)
    
    # YOUR CODE STARTS HERE
    
    parameters = initialize_parameters_deep(layers_dims)
    
    # YOUR CODE ENDS HERE
    
    # Loop (gradient descent)
    for i in range(0, num_iterations):

        # Forward propagation: [LINEAR -> tanh]*(L-1) -> LINEAR -> SIGMOID.
        #(≈ 1 line of code)
        
        # YOUR CODE STARTS HERE
        
        AL, caches = L_model_forward(X, parameters, ss)
        
        # YOUR CODE ENDS HERE
        
        # Compute cost.
        #(≈ 1 line of code)
    
        # YOUR CODE STARTS HERE
        
        cost = compute_cost(AL, Y)
        
        # YOUR CODE ENDS HERE
    
        # Backward propagation.
        #(≈ 1 line of code)
        
        # YOUR CODE STARTS HERE
        
        grads = L_model_backward(AL, Y, caches)
        
        # YOUR CODE ENDS HERE
 
        # Update parameters.
        #(≈ 1 line of code)
        
        # YOUR CODE STARTS HERE
        
        parameters = update_parameters(parameters, grads, learning_rate)
        
        # YOUR CODE ENDS HERE
                
        # Print the cost every 100 iterations
        if print_cost and i % 100 == 0 or i == num_iterations - 1:
            print("Cost after iteration {}: {}".format(i, np.squeeze(cost)))
        if i % 100 == 0 or i == num_iterations:
            costs.append(cost)
    
    return parameters, costs

In [28]:
parameters, costs = L_layer_model(X, Y, layers_dims, num_iterations = 10000, print_cost = True, ss = True)

Cost after iteration 0: 0.6931471805601178
Cost after iteration 100: 0.6930252636961056
Cost after iteration 200: 0.6930244925332523
Cost after iteration 300: 0.6930244876532351
Cost after iteration 400: 0.6930244876223416
Cost after iteration 500: 0.6930244876221351
Cost after iteration 600: 0.6930244876221229
Cost after iteration 700: 0.693024487622112
Cost after iteration 800: 0.693024487622101
Cost after iteration 900: 0.6930244876220899
Cost after iteration 1000: 0.6930244876220789
Cost after iteration 1100: 0.6930244876220681
Cost after iteration 1200: 0.693024487622057
Cost after iteration 1300: 0.693024487622046
Cost after iteration 1400: 0.6930244876220352
Cost after iteration 1500: 0.6930244876220241
Cost after iteration 1600: 0.6930244876220132
Cost after iteration 1700: 0.6930244876220022
Cost after iteration 1800: 0.6930244876219912
Cost after iteration 1900: 0.6930244876219803
Cost after iteration 2000: 0.6930244876219693
Cost after iteration 2100: 0.6930244876219582
Cost

In [26]:
def predict(X, Y, parameters, ss):
    """
    Using the learned parameters, predicts a class for each example in X
    
    Arguments:
    parameters -- python dictionary containing your parameters 
    X -- input data of size (n_x, m)
    
    Returns
    predictions -- vector of predictions of our model (0 / 1)
    """
    
    A, cache = L_model_forward(X, parameters, ss)
    
    incorrect = (A > 0.5).astype(int) == Y
    sum = 0
    
    for i in range (incorrect.shape[1]):
        sum = sum + incorrect[0][i]
    
    return sum/Y.shape[1]

In [27]:
p = predict(X,Y,parameters, True)
print(p)

0.5078322385042952
